In [1]:
import numpy as np
import pandas as pd
#import statsmodels.api as sm
import databricks.koalas as ks
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import time
#import datetime


In [6]:
# instalar java jdk ---> sudo apt-get install -y openjdk-8-jdk-headless 
import os       #importing os to set environment variable
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     #set environment variable

### Cargo los parquet

In [2]:
#raw_data_act = ks.read_parquet('activityObfmod5.parquet',columns=['user_id','pediodo'])
#raw_data_sb = ks.read_parquet('sbObfmod5.parquet')

raw_data_act = pd.read_parquet('activityObfmod5.parquet')
raw_data_sb = pd.read_parquet('sbObfmod5.parquet')

In [11]:
raw_data_act.head()

,user_id,periodo
0,196287654008258688,2015-02-01
1,196287654008258688,2015-03-01
2,196287654008258688,2015-04-01
3,196287654008258688,2015-05-01
4,196287654008258688,2015-06-01


### Funciones

In [2]:
def hist_actividad(df, feature_name, year, histogram = True):
    '''Si histogram == False, devuelve un dataframe con los períodos de actividad y user_id de determinado año'''
    #aux = df[(df[feature_name] >= datetime.date(year,1,1)) & (df[feature_name] < datetime.date(year+1,1,1))]  
    aux = df[(df[feature_name] >= '{}-1-1'.format(year) ) & (df[feature_name] < '{}-1-1'.format(year+1) )] 
    
    if histogram == True:
        plt.hist(aux[feature_name]);        
        plt.title(year, fontsize = 20)
    
    else:
        return aux

def get_month_diff(a, b):
      
      return 12*(a.year - b.year) + (a.month - b.month)       
      
def se_fuga(df):
    
      if (df['user_id'] == df['next_user_id']) and ( get_month_diff(df['next_periodo'], df['periodo']) >= 4) :
          return 1
      #elif (df['user_id'] != df['next_user_id']) and ( get_month_diff(max_date, df['periodo']) >= 4) :
      #    return 0
      else:
          return 0
    
def define_estados(df_aux):
    
      df2 = df_aux.copy()
      
      df2['next_user_id'] = np.array(df2['user_id'].shift(-1))
      df2['next_periodo'] = np.array(df2['periodo'].shift(-1))
      
      return df2.apply(se_fuga, axis=1)
    
    

### Análisis de datos

In [4]:
df_act_sorted=raw_data_act.sort_values(by=['user_id','periodo']).reset_index(drop=True) # Ordeno el dataframe de Activity
df_act_wo_duplicates = df_act_sorted.drop_duplicates(keep = 'last').reset_index(drop=True) # Saco las filas repetidas

#max_date = df_act_wo_duplicates['periodo'].max() # Fecha más reciente registrada

In [5]:
df_act_wo_duplicates.describe()

,user_id,periodo
count,18267701,18267701
unique,470794,80
top,6904498029018699,2018-07-01
freq,58,366347


##### Número de usuarios:  470794

In [17]:
df_act_wo_duplicates.head()

,user_id,periodo
0,100000586751768816,2015-02-01
1,100000586751768816,2015-03-01
2,100000586751768816,2015-04-01
3,100000586751768816,2015-05-01
4,100000586751768816,2015-06-01


In [6]:
start_time = time.clock()

states = define_estados(df_act_wo_duplicates);
print((time.clock() - start_time)/60, "minutos")

/home/dran/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  This is separate from the ipykernel package so we can avoid doing imports until


7.478314616666667 minutos


/home/dran/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  import sys


In [7]:
df_act_wo_duplicates['Estado'] = states
df_act_wo_duplicates.to_csv(r'df_act_wo_duplicates.csv')


In [10]:
#df_act_wo_duplicates.to_csv(r'df_act_wo_duplicates.csv')
df_act_wo_duplicates[(df_act_wo_duplicates['Estado']==1) & (df_act_wo_duplicates['user_id'] == df_act_wo_duplicates['user_id'].shift(-1)) ]

,user_id,periodo,Estado
594,100012779129386160,2014-06-01,1
1858,100054053520973648,2015-10-01,1
2001,100055557365138304,2014-06-01,1
2196,100062857068564480,2014-08-01,1
3069,10007901558144382,2014-06-01,1
3639,100090300784319584,2016-07-01,1
3695,100093183741879664,2014-06-01,1
4146,100105636671398832,2014-07-01,1
5135,100123764934496656,2014-06-01,1
6371,100146953636890880,2014-07-01,1


In [4]:
df_sb_sorted = raw_data_sb.sort_values(by=['user_id','periodo']).reset_index(drop=True) # Ordeno el dataframe sb
df_sb_wo_duplicates = df_sb_sorted.drop_duplicates(keep = 'last').reset_index(drop=True) # Quito filas duplicadas


In [5]:
df_sb_wo_duplicates.to_csv(r'df_sb_wo_duplicates.csv')

In [3]:
## CARGAR DATOS act Y sb SIN DUPLICADOS Y ORDENADOS

df_sb_wo_duplicates = pd.read_csv('df_sb_wo_duplicates.csv')
df_act_wo_duplicates  = pd.read_csv('df_act_wo_duplicates.csv')

df_sb_wo_duplicates = df_sb_wo_duplicates.drop(columns=['Unnamed: 0'])
df_act_wo_duplicates = df_act_wo_duplicates.drop(columns=['Unnamed: 0'])

#### Acá hice una función que devuelve cuántos períodos de actividad hay por user_id, y cuántas filas de datos hay para ese user en el otro data set. Haciendo un vistazo rápido, se ve que si el número de actividades por usuario es mayor que su cantidad de datos demográficos, entonces hay datos que faltan. 

In [4]:
def cuenta_actividades(users):
  
    Num_actividades = np.zeros(len(users))
    Num_datos_demog = np.zeros(len(users))
    
    for i, userid in enumerate(users):

      Num_actividades[i] = len(df_act_wo_duplicates[df_act_wo_duplicates['user_id']== userid])
      Num_datos_demog[i] = len(df_sb_wo_duplicates[df_sb_wo_duplicates['user_id'] == userid])

    
    df_users = pd.DataFrame()
    df_users['User_id'] = users
    df_users['Cantidad_actividades'] = Num_actividades
    df_users['Cantidad_filas_datos_demog'] = Num_datos_demog
    
    print(np.sum(df_users['Cantidad_filas_datos_demog']))
    return df_users
  

def merge_act_sb(df_act, df_sb):
  
  index_sb = []; estado_act = [];
   
  for i in range(df_act.shape[0]):
    
      if len(df_sb[(df_sb['periodo'] == df_act['periodo'][i]) & (df_sb['user_id'] == df_act['user_id'][i]) ]) > 0:
          
          index_aux = df_sb[(df_sb['periodo'] == df_act['periodo'][i]) & (df_sb['user_id'] == df_act['user_id'][i])].index[0]
          index_sb.append(index_aux)
          estado_act.append(df_act['Estado'][i])
  
  df_sb_act = df_sb.iloc[index_sb]
  df_sb_act['estado'] = estado_act
  df_sb_act.reset_index(drop=True)
  
  return df_sb_act
    
  
  
  

In [0]:

start_time = time.clock()

df_sb_act = merge_act_sb(df_act_wo_duplicates, df_sb_wo_duplicates)

df_sb_acr.to_csv(r'/content/drive/My Drive/Mis colabs/df_sb_act.csv')

print(time.clock() - stat_time, "segundos")

df_sb_act.head(20)

In [ ]:
users_array = df_act_wo_duplicates['user_id'].unique()  # array con los id usuarios
  
cantidad_act_sb=cuenta_actividades(users_array)
cantidad_act_sb.head(10)

# Resources
##Apache Spark
Framework para procesamiento distribuido.
doc: https://spark.apache.org/

##Koalas

Una libreria del estilo Pandas  que esa spark por atras para el manejo de datos.
https://koalas.readthedocs.io/en/latest/

## scikit-learn
Una libreria para machine learning

https://scikit-learn.org/stable/

##cursos copados

los cursos estos son de gran ayuda en general, **no es para que los veas ahora**, son lindos recursos para meterse de lleno en el mundo del data scientist dev.

###ML

https://www.coursera.org/courses?query=machine+learning+andrew+ng

https://www.coursera.org/specializations/deep-learning

https://www.udemy.com/machinelearning-es/

###DS

https://www.coursera.org/specializations/data-science-python

###Scala

https://www.coursera.org/specializations/scala

https://www.udemy.com/the-ultimate-hands-on-hadoop-tame-your-big-data/




Libros hay millones si queres despues te paso un repo donde tengo una cantidad infernal, dependiendo el tema.



